# SOLUTION SHEET: Hands-On Big Data Processing with MapReduce

This notebook contains the complete solutions for the MapReduce workshop exercises.

### Setup

First, let's make sure you have the necessary libraries and data. Run the cell below to download the text for *Moby Dick*.

In [1]:
# Let's import the libraries we'll need for this workshop
import time
import re
from collections import Counter, defaultdict
import multiprocessing
import matplotlib.pyplot as plt
import requests
import os

# Download the file if it doesn't exist
file_path = 'moby_dick.txt'
if not os.path.exists(file_path):
    print(f"Downloading Moby Dick to {file_path}...")
    url = 'https://www.gutenberg.org/ebooks/2701.txt.utf-8'
    try:
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print("Download complete.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
else:
    print(f"{file_path} already exists.")

moby_dick.txt already exists.


---

## Exercise 1: Baseline Performance - Serial Word Count

**Task:** Before optimizing, we need a baseline. Complete the `serial_word_count` function below.

In [2]:
def serial_word_count(file_path):
    """
    Reads a text file and counts the frequency of words in a serial manner.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read().lower()
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Please run the setup cell first.")
        return None

    # SOLUTION: Find all words using a regular expression.
    # The pattern r'\b[a-z]{3,}\b' is a good starting point.
    words = re.findall(r'\b[a-z]{3,}\b', text)

    # SOLUTION: Use collections.Counter to count the words.
    word_counts = Counter(words)

    return word_counts

# --- Execution and Timing ---
print("Starting serial word count...")
start_time_serial = time.perf_counter()
s_word_counts = serial_word_count(file_path)
end_time_serial = time.perf_counter()

serial_execution_time = end_time_serial - start_time_serial

print(f"Serial execution took: {serial_execution_time:.4f} seconds")
# Optional: Print the 10 most common words to verify
if s_word_counts:
    print("10 most common words:", s_word_counts.most_common(10))

Starting serial word count...
Serial execution took: 0.1675 seconds
10 most common words: [('the', 14715), ('and', 6514), ('that', 3081), ('his', 2530), ('but', 1822), ('with', 1769), ('was', 1647), ('for', 1644), ('all', 1543), ('this', 1437)]


---

## Exercise 2: Parallel Processing with MapReduce

**Task:** Now, implement the same word count logic using a MapReduce approach.

### 2.1: Prepare Data Chunks
First, we need to split our data into chunks to be processed in parallel.

In [3]:
# Determine the number of processes to use (usually the number of CPU cores)
# You can change this number to see how it affects performance!
NUM_PROCESSES = multiprocessing.cpu_count()
print(f"Using {NUM_PROCESSES} processes.")

def chunkify_text(file_path, num_chunks):
    """Splits the text into a specified number of chunks."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        chunk_size = len(text) // num_chunks
        chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
        return chunks
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
        return []

# Create the chunks for our mappers
text_chunks = chunkify_text(file_path, NUM_PROCESSES)
if text_chunks:
    print(f"Split text into {len(text_chunks)} chunks.")

Using 16 processes.
Split text into 17 chunks.


### 2.2: Implement the `Mapper` and `Reducer`

**Task:** Complete the `mapper` and `reducer` functions below.

In [8]:
from mapreduce_functions import mapper, reducer

In [ ]:
def mapper(text_chunk):
    """
    Processes a chunk of text, finds words, and returns a list of (word, 1) tuples.
    """
    # SOLUTION: Convert the chunk to lowercase and find all words.
    text_chunk = text_chunk.lower()
    words = re.findall(r'\b[a-z]{3,}\b', text_chunk) 
    
    # SOLUTION: Create a list of (word, 1) tuples and return it.
    mapped_results = [(word, 1) for word in words]
    return mapped_results


def reducer(item):
    """
    Reduces a key and its list of values to a single value.
    Takes one item from the shuffled list, e.g., ('whale', [1, 1, 1, 1]).
    """
    word, counts = item
    # SOLUTION: Return a tuple of the word and the sum of its counts.
    return (word, sum(counts))

### 2.3: Orchestrate Map, Shuffle, and Reduce

**Task:** Now, put all the pieces together.

In [ ]:
# --- Initialization ---
# Initialize an empty dictionary to store the final word counts from the MapReduce process.
mr_word_counts = {}
# Initialize a variable to hold the total execution time. We set it to 0 in case the process is skipped.
mapreduce_execution_time = 0

# --- Safety Check ---
# This 'if' statement is a safety check. It ensures that the MapReduce logic only runs
# if the 'text_chunks' list was successfully created in the previous step.
# If the file wasn't found or was empty, text_chunks would be empty or None, and this block would be skipped.
if text_chunks:
    print("Starting MapReduce word count...")
    # Record the starting time using a high-precision performance counter.
    start_time_mapreduce = time.perf_counter()

    # --------------------------------------------------------------------------
    # --- MAP STAGE: Process data chunks in parallel ---
    # --------------------------------------------------------------------------
    # A 'Pool' is an object that manages a pool of worker processes.
    # 'NUM_PROCESSES' is typically set to the number of CPU cores on the machine.
    # The 'with' statement ensures that all processes in the pool are properly closed down
    # once the work is done, even if errors occur.
    with multiprocessing.Pool(processes=NUM_PROCESSES) as pool:
        # 'pool.map()' is the core of the parallel execution. It does three things:
        # 1. It takes the iterable 'text_chunks' and splits it up among the worker processes in the pool.
        # 2. It applies the 'mapper' function to each individual chunk in parallel.
        # 3. It waits for all workers to finish and collects their results into a single list.
        # The result, 'mapped_results', will be a list of lists, e.g., [[('the', 1), ('a', 1)], [('whale', 1), ('the', 1)], ...]
        mapped_results = pool.map(mapper, text_chunks)

    # --------------------------------------------------------------------------
    # --- SHUFFLE STAGE: Group the results by key (word) ---
    # --------------------------------------------------------------------------
    # 'defaultdict(list)' is a convenient type of dictionary. If you try to access or add to a key
    # that doesn't exist, it automatically creates that key with a default value, which is an empty list [] in this case.
    # This is perfect for grouping items without needing to check if the key exists first.
    shuffled_data = defaultdict(list)
    
    # This nested loop iterates through the 'mapped_results' to group them.
    # The outer loop goes through each list returned by a mapper process (e.g., [('whale', 1), ('the', 1)]).
    for result_list in mapped_results:
        # The inner loop goes through each (key, value) tuple in that list (e.g., ('whale', 1)).
        for key, value in result_list:
            # For each key (the word), it appends the value (the number 1) to its list in the defaultdict.
            # After this loop, shuffled_data will look like: {'the': [1, 1, 1,...], 'whale': [1, 1,...], ...}
            shuffled_data[key].append(value)

    # --------------------------------------------------------------------------
    # --- REDUCE STAGE: Aggregate the grouped values in parallel ---
    # --------------------------------------------------------------------------
    # We create another pool of worker processes for the reduce step.
    with multiprocessing.Pool(processes=NUM_PROCESSES) as pool:
        # We again use 'pool.map()'. This time, we apply the 'reducer' function.
        # 'shuffled_data.items()' provides an iterable of (key, value_list) pairs,
        # for example: ('the', [1, 1, 1, ...]).
        # Each of these items is sent to a worker process to be "reduced" (in this case, summed).
        reduced_results = pool.map(reducer, shuffled_data.items())

    # --------------------------------------------------------------------------
    # --- Finalization ---
    # --------------------------------------------------------------------------
    # 'reduced_results' is currently a list of tuples, e.g., [('the', 14715), ('whale', 1234), ...].
    # The 'dict()' constructor efficiently converts this list of key-value pairs into a Python dictionary.
    mr_word_counts = dict(reduced_results)

    # Record the ending time.
    end_time_mapreduce = time.perf_counter()
    # Calculate the total duration of the MapReduce process.
    mapreduce_execution_time = end_time_mapreduce - start_time_mapreduce

    print(f"MapReduce execution took: {mapreduce_execution_time:.4f} seconds")

    # This is an optional but important verification step.
    if mr_word_counts:
        # We sort the final dictionary by its values (the word counts) in descending order to find the most frequent words.
        # 'sorted()' returns a list of (key, value) tuples.
        # 'key=lambda item: item[1]' tells the sort function to use the second element of each tuple (the count) for sorting.
        sorted_counts = sorted(mr_word_counts.items(), key=lambda item: item[1], reverse=True)
        # We print the top 10 results to visually confirm they are sensible and likely match the serial version's output.
        print("10 most common words:", sorted_counts[:10])
else:
    # This message is printed if the initial 'if text_chunks:' check failed.
    print("Skipping MapReduce execution because data chunks were not created.")

Starting MapReduce word count...


Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'mapper' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/

KeyboardInterrupt: 

---

## Exercise 3: Performance Comparison and Visualization

**Task:** Now that you have timings for both methods, create a bar chart to visually compare them.

In [ ]:
# Data for plotting
methods = ['Serial Processing', f'MapReduce ({NUM_PROCESSES} Cores)']
times = [serial_execution_time, mapreduce_execution_time]

# Create the bar plot
plt.figure(figsize=(8, 6))
bars = plt.bar(methods, times, color=['skyblue', 'lightgreen'])
plt.ylabel('Execution Time (seconds)')
plt.title('Performance Comparison: Serial vs. MapReduce')

# Add the time values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.4f}s', va='bottom', ha='center')

plt.show()

# Print the speedup factor
if mapreduce_execution_time > 0:
    speedup = serial_execution_time / mapreduce_execution_time
    print(f"\nMapReduce was {speedup:.2f}x faster than the serial implementation.")

---

## Exercise 4: Conceptual Questions on Hadoop

**Task:** Based on your reading from Part B of the handout, answer the following questions.

### Questions & Solutions:

**1. What is the Hadoop Distributed File System (HDFS)? What problem does it solve?**

*__Solution:__ HDFS is a specialized filesystem designed to store and manage massive datasets (terabytes or petabytes) across a large cluster of commodity (standard, inexpensive) computers. It solves the problem of storing data that is too large to fit on a single machine. It does this by splitting large files into smaller blocks and distributing those blocks across many computers in the cluster. It also provides fault tolerance by replicating each block multiple times on different machines, so if one machine fails, the data is not lost.*

**2. What are the roles of the `NameNode` and `DataNode` in HDFS?**

*__Solution:__ HDFS has a master/worker architecture:
- **NameNode (Master):** The NameNode is the brain of the filesystem. It does not store the actual data itself. Instead, it holds the metadata, which is like a table of contents. This metadata includes the file directory structure, permissions, and, most importantly, the location of all the data blocks for every file in the cluster. All requests to read or write a file go to the NameNode first.
- **DataNode (Worker):** DataNodes are the workhorses. They are responsible for storing the actual data blocks on their local disks. They report back to the NameNode periodically with a list of the blocks they are storing to prove they are alive and well (this is called a "heartbeat").*

**3. What is YARN (Yet Another Resource Negotiator)? What is its role in the Hadoop ecosystem?**

*__Solution:__ YARN is the resource management and job scheduling layer of Hadoop. It acts as the "operating system" for the entire Hadoop cluster. Its primary role is to manage the cluster's resources (CPU, memory) and allocate them to various applications. Before YARN, MapReduce was tightly coupled with resource management. YARN separated these, allowing different data processing frameworks (like Apache Spark, Flink, and MapReduce itself) to run simultaneously on the same Hadoop cluster, sharing resources efficiently.*

**4. How does running MapReduce on Hadoop differ from our simple Python implementation?**

*__Solution:__ There are several key differences:
1. **Scale:** Our Python script runs on a single machine, limited by its CPU cores and RAM. Hadoop runs on a cluster of tens to thousands of machines, enabling analysis of vastly larger datasets.
2. **Data Storage:** Our script reads from a local file. Hadoop's MapReduce reads data directly from HDFS, where the data is already distributed across the cluster.
3. **Fault Tolerance:** If our Python script fails (or the computer crashes), the job stops and must be restarted manually. Hadoop is designed for failure; if a DataNode or a specific task fails, YARN will automatically reschedule the task on another available machine.
4. **Data Locality:** Our script moves data from the disk to the CPU. Hadoop is built on the principle of moving the *computation to the data*. YARN tries to schedule MapReduce tasks on the same DataNodes where the data blocks reside, minimizing network traffic and drastically improving efficiency.*

---

## Exercise 5: Looking Ahead - Final Thoughts

**Task:** In the markdown cell below, briefly brainstorm one or two ideas for how you could use the MapReduce pattern in your next major assignment.

### Example Ideas for the Next Assignment:

**Dataset Idea:** *A large dataset of millions of e-commerce customer reviews.*

**MapReduce Application:** *The goal is to find the average rating and number of reviews for every unique product ID. A serial loop would be very slow.
- **Map:** The mapper would process each row (review). For each review, it would emit a key-value pair of `(product_id, (rating, 1))`. The rating is the score, and the 1 is to count the review.
- **Reduce:** The reducer would receive a product ID and a list of tuples, e.g., `(product_123, [(5, 1), (4, 1), (5, 1)])`. It would then calculate the sum of ratings and the sum of the counts to produce a final output: `(product_123, (average_rating, total_reviews))`. This pre-computed data makes visualizing top/bottom products much faster.*

---

**Dataset Idea:** *A year's worth of server access logs from a high-traffic website (billions of lines).*

**MapReduce Application:** *The goal is to identify which IP addresses are accessing the site most frequently, potentially to detect bots or scraping activity.
- **Map:** The mapper would parse each log line. It would extract the client IP address and emit a key-value pair of `(ip_address, 1)`.
- **Reduce:** The reducer would receive an IP address and a list of ones, e.g., `(192.168.1.1, [1, 1, 1, ...])`. It would simply sum the list to get the total number of hits for that IP. The final output can then be sorted to find the top IP addresses.*